## **PROJECT 1**

### **'다음 분기에 어떤 게임을 설계해야하는가?'**




#### **주요 분석**

> * `지역에 따라 선호하는 게임 장르가 다른지`
* `연도별 게임 트렌드가 존재하는지`
* `출고량이 높은 게임에 대해 분석 및 시각화`






### **1.데이터 전처리**

In [ ]:
# 데이터 불러오기

import pandas as pd

df = pd.read_csv('/content/vgames2.csv')
df

**EDA (Exploratory Data Analysis)**

In [ ]:
df.dtypes

#각 지역 출고량이 object형태임을 확인

In [ ]:
df = df.drop(['Unnamed: 0'],axis=1) #불필요한 컬럼 제거

In [ ]:
 df.isnull().sum() #결측값 확인

In [ ]:
df1 = df.dropna(axis=0) #결측값 제거
df1 = df1.reset_index(drop = True) #Reindex
df1

In [ ]:
#각 지역 출고량 형변환
def sales(x) :
  if x[-1:] == 'K' :
    x1 = pd.to_numeric(x[:-1]) * 1000
    return x1
  elif x[-1:] == 'M' :
    x1 = pd.to_numeric(x[:-1]) * 1000000
    return x1
  else :
    x1 = pd.to_numeric(x) * 1000000
    return x1

In [ ]:
#함수 적용
df1['NA_Sales']=df1['NA_Sales'].apply(sales)
df1['EU_Sales']=df1['EU_Sales'].apply(sales)
df1['JP_Sales']=df1['JP_Sales'].apply(sales)
df1['Other_Sales']=df1['Other_Sales'].apply(sales)

In [ ]:
import matplotlib.pyplot as plt

#지역 별 출고량 분포

plt.figure(figsize=(10,5))
plt.boxplot([df1['NA_Sales'],df1['EU_Sales'],df1['JP_Sales'],df1['Other_Sales']])
plt.xticks([1, 2, 3, 4], ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']) #label 붙이기
plt.show()

0~20000000 사이에 밀집되어 분포하고 있다.


In [ ]:
#year 변환
def year(x):
    if len(str(int(x))) == 1:
        x = x+2000
    elif len(str(int(x))) == 2:
        if int(x) <= 25:
            x = x+2000
        else:
            x = x+1900
    return x

#함수 적용
df1['Year'] = df1['Year'].apply(year)
df1['Year']

In [ ]:
plt.hist(df1['Year'])
plt.title('Year histogram')
plt.show()

###**2.분석 및 결과**

###**1) 분석**
*   지역에 따라 선호하는 게임 장르가 다른가?


PCA 실행

> : 장르별로 각 지역에 대해 게임 출고량은 어떠한 관계를 보이는가

In [ ]:
#장르 별 합계
na = df1.groupby('Genre')['NA_Sales'].sum()
eu = df1.groupby('Genre')['EU_Sales'].sum()
jp = df1.groupby('Genre')['JP_Sales'].sum()
other = df1.groupby('Genre')['Other_Sales'].sum()



sum_genre = pd.concat([na,eu,jp,other],axis=1).T.reindex()
sum_genre

In [ ]:
#표준점수 구하기
from sklearn.preprocessing import StandardScaler
x = sum_genre.values # 독립변인들의 value값만 추출
y = sum_genre.index # 종속변인 추출

x = StandardScaler().fit_transform(x) # x객체에 x를 표준화한 데이터를 저장

features = ['Action',	'Adventure',	'Fighting',	'Misc',	'Platform',	'Puzzle',	'Racing',	'Role-Playing',	'Shooter',	'Simulation',	'Sports',	'Strategy']
pd.DataFrame(x, columns=features)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2) # 주성분 개수 결정
printcipalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data=printcipalComponents, columns = ['PC1', 'PC2'])

principalDf # 주성분으로 이루어진 데이터 프레임 구성

In [ ]:
d_genre= pd.DataFrame({'local':['NA_Sales','EU_Sales','JP_Sales','Other_Sales']})

finalDf = pd.concat([principalDf,d_genre],axis=1)

finalDf

In [ ]:
pca.explained_variance_ratio_

In [ ]:
import matplotlib.pyplot as plt


fig = plt.figure(figsize = (8, 8))
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('PCA', fontsize=20)

targets = ['NA_Sales',	'EU_Sales',	'JP_Sales',	'Other_Sales']

for target, color in zip(targets,colors):
    indicesToKeep = finalDf['local'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'PC1']
               , finalDf.loc[indicesToKeep, 'PC2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

Japan을 제외하고 나머지는 유사하다고 볼 수 있다.

In [ ]:
#장르 별 평균 집계
df_na = df1.groupby('Genre')['NA_Sales'].mean()
df_eu = df1.groupby('Genre')['EU_Sales'].mean()
df_jp = df1.groupby('Genre')['JP_Sales'].mean()
df_other = df1.groupby('Genre')['Other_Sales'].mean()


df_by_genre = pd.concat([df_na,df_eu,df_jp,df_other], axis=1)
df_by_genre

In [ ]:
from scipy import stats


In [ ]:
df_by_genre.plot(kind='line',figsize = (15,10), rot=0)

plt.title('Aggregation by genre')

plt.legend(fontsize='13')

plt.show()


Japan 을 제외한 나머지 지역들은 패턴의 유사성을 갖고 있다.




따라서, Japan을 제외한 나머지 지역들의 선호하는 게임 장르는 유사하다.











*   연도별 게임의 트렌드가 존재하는가?




In [ ]:
df1['total'] = df1['NA_Sales'] + df1['JP_Sales'] + df1['EU_Sales'] + df1['Other_Sales'] #total 게임 출고량 

In [ ]:
#1. 연도별 게임 총 출고량
df_year = df1.groupby('Year')['total'].sum()

#시각화
plt.plot(df_year)
plt.title('Game Total Shipment by Yearly')
plt.show()

In [ ]:
#2. 연도별 게임 장르의 트렌드
year_genre = df1.groupby(['Year','Genre']).size().reset_index(name='count')
year_max = year_genre.groupby(['Year'])['count'].transform(max) == year_genre['count']
year_max_genre = year_genre[year_max].reset_index(drop=True)

#중복값 제외
year_max_genre = year_max_genre.drop_duplicates(subset=['Year','count'], keep='last').reset_index(drop=True)
year_max_genre[:5]
year_max_genre.columns = ['Year','Genre','Sheepment']
year_max_genre

In [ ]:
#시각화
import seaborn as sns

plt.figure(figsize = (25,15))
ax = sns.barplot(x='Year', y='Sheepment', data=year_max_genre)
idx = 0
for value in year_max_genre['Sheepment']:
  ax.text(x=idx,y=value+5, s=str(year_max_genre['Genre'].values[idx]),size=14,rotation=90,ha='center')
  idx += 1
plt.xlabel('Year', fontsize=15)
plt.ylabel('Sheepment', fontsize=15)
ax.set_title('Genre with Top Annual Releases')
plt.show()

장르 중 Action이 최신 트랜드라고 볼 수 있다.


* 출고량이 높은 게임에 대해 분석 및 시각화







In [ ]:
df3 = df1.copy()
df3.index = df3['Name']

# top3
print('전체 출고량 top3 : \n', df3['total'].nlargest(3)) 
print('\n최근 출고량 top3 : \n', df3[df3['Year'] >= 2015]['total'].nlargest(3))

In [ ]:
#최근 장르별 순위
import seaborn as sns

trend = df3[df3['Year'] >= 2015].iloc[:100]['Genre'].value_counts()
trend = pd.DataFrame(trend)
trend.reset_index(inplace = True)
trend.columns = ['Genre', 'count']

trend

In [ ]:
plt.figure(figsize=(11,8))

sns.barplot(x = 'Genre', y = 'count', data = trend, hue = 'Genre').set(title = "Top Genre Rankings Reflecting Recent Trends")

In [ ]:
#최근 장르별 출고량 평균 순위
Genre_total_trend = pd.DataFrame(df3[df3['Year']>=2015].groupby('Genre')['total'].mean())
Genre_total_trend.reset_index(inplace = True)
Genre_total_trend.sort_values('total', ascending = False, inplace = True)

Genre_total_trend

In [ ]:
#시각화
plt.figure(figsize = (12,8))

sns.barplot(x='Genre', y='total', data=Genre_total_trend).set(title='Release Average Rank by Latest Genre')

ANOVA 


> Shooter, Sports, Role-Playing 과연 출고량에서 유의미한 차이가 있을까?



In [ ]:
# 출고량 top 3 Genre 평균 비교
from scipy import stats
trend2 = df3[df3['Year'] >= 2015]

group1 = trend2[trend2['Genre'] == 'Shooter']['total']
group2 = trend2[trend2['Genre'] == 'Sports']['total']
group3 = trend2[trend2['Genre'] == 'Role-Playing']['total']

#Shooter, Sports , Role-playing ANOVA
p_value_ano = stats.f_oneway(group1, group2, group3).pvalue

if p_value_ano < 0.05:
    print('Shooter, Sports, Role-Playing 세 그룹간 평균이 적어도 하나는 유의미하게 차이납니다.')
else:
    print('Shooter, Sports, Role-Playing 세 그룹간 평균이 유의미하게 차이나지 않습니다.')

In [ ]:
#등분산 test
p_value_var = stats.levene(group1, group2).pvalue

if p_value_var >= 0.05:
    print('Shooter, Sports 두 그룹은 등분산이다.')
else:
    print('Shooter, Sports 두 그룹은 등분산이 아니다.')

#독립 t-test, 단측검정:
p_value_t = stats.ttest_ind(group1, group2, equal_var = False).pvalue

if p_value_t >= 0.05:
    print('Shooter, Sports 두 그룹간 평균이 유의미하게 차이나지 않습니다.')
else:
    print('Shooter, Sports 두 그룹간 평균이 Shooter가 더 크다.')

In [ ]:
#최근 shooter를 통한 CI 계산
import numpy as np
avg = np.mean(group1)
se = np.std(group1)/np.sqrt(len(group1))

ci = stats.t.interval(.95, loc = avg, scale = se, df = len(group1)-1)

print('Shooter 평균 출고량의 confidence interval : ', (round(ci[0],3), round(ci[1], 3))) 

In [ ]:
# 최근 평균 출고량이 가장 높은 플랫폼

Platform_total_trend = pd.DataFrame(df3[df3['Year']>=2015].groupby('Platform')['total'].mean())
Platform_total_trend.reset_index(inplace = True)
Platform_total_trend.sort_values('total', ascending = False, inplace = True)

plt.figure(figsize = (10,6))

sns.barplot(x='Platform', y='total', data=Platform_total_trend)
plt.title('Platform with the highest average shipments recently')

In [ ]:
#PS4에서 shooter 게임을 잘 만드는지

#PS4에서 만든 많이 만든 장르 순위
PS4 = df3[df3['Platform']=='PS4']

PS4['Genre'].value_counts()

In [ ]:
#PS4 회사에서 만든 장르별 총 출고량 순위
PS4.groupby('Genre')['total'].sum().sort_values(ascending=False)

In [ ]:
#PS4에서 만든 장르별 총 출고량 평균 순위
PS4.groupby('Genre')['total'].mean().sort_values(ascending=False)

In [ ]:
#shooter을 가장 많이 출고한 platform

shooter_platform = pd.DataFrame(df3[df3['Genre'] == 'Shooter'].groupby('Platform')['total'].sum().sort_values(ascending=False))
shooter_platform.reset_index(inplace = True)

shooter_platform

In [ ]:
#시각화
plt.figure(figsize=(10,6))

sns.barplot(x='Platform', y='total', data=shooter_platform).set(title='The platform with the most Shooter shipped')

**독립성 검정(T-test)** : 관측값들이 다수의 인자들에 의해 분할 되어 있는 경우 그 인자들의 관찰 값에 영향을 주고 있는지 아닌지를 검정하는 방법


* 귀무가설 : 독립이다. (연관성이 없다)
* 대립가설 : 독립이 아니다.



In [ ]:
#장르 & 플랫폼 간의 연관성
cross_chi  = df3[['Genre', 'Platform','total']]

cross_chi

In [ ]:
cross_chi2 = pd.crosstab(cross_chi['Genre'], cross_chi['Platform'])

cross_chi2

In [ ]:
p_value_chi2 = stats.chi2_contingency(cross_chi2)[1]

if p_value_chi2 < 0.05:
    print('Genre와 Platform 사이의 연관성이 존재하지 않는다.(귀무가설 채택)')
else:
    print('Genre와 Platform 사이의 연관성이 존재한다.(대립가설 채택)')

장르와 플랫폼은 서로 독립적인 관계로 *'특정 플랫폼이 특정 장르를 독점하고 있지 않다'* 라고 볼 수 있다.


1. 즉, 전체 플랫폼이 shooter에 대해 지원가능하며
2. 이 중에서도 X360이 가장 높은 출고량을 갖는다.



###**2) 결과**




*  Shooter 게임 위주
*  X360 회사와의 콜라보레이션으로 런칭
*  일본을 제외한 모든 나라에 수출
*  이후 일본을 타겟으로 하여 마케팅 시도
*  빠른 출시






